# Abstract

# Preface

## Aim

To create a Bayes Learning model to predict the scientific abstract based on proteins present

## Method



## Results


## Conclusions

# Code

## Get all required imports

In [2]:
import pandas as pd
from pandas import DataFrame
import nltk
from math import log

nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist

stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\liyor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\liyor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Load Data

In [3]:
# abstractTrainingData (absTrainData)
absTrainData = pd.read_csv("data/trg.csv")
absTestData  = pd.read_csv("data/tst.csv")

absTrainData = absTrainData.drop(columns="id")
absTestData  = absTestData.drop(columns="id")


classDic = {"A": "Archaea", "B": "Bacteria", "E": "Eucaryota", "V": "Virus"}

classKeys = list(classDic)


# Check variables to see if its as expected
print(absTrainData)
print(absTestData)
print(classDic)
print(classKeys)


     class                                           abstract
0        B  the 4 202 353 bp genome of the alkaliphilic ba...
1        A  the complete 1751377-bp sequence of the genome...
2        E  in 1992 we started assembling an ordered libra...
3        E  the aim of this study is to measure human mito...
4        B  the amino acid sequence of the spirulina maxim...
...    ...                                                ...
3995     E  we have isolated and characterized two diureti...
3996     E  myotonias are muscle diseases in which the fun...
3997     E  cysteine synthase o-acetylserine sulfhydrylase...
3998     E  a region of 25 nucleotides is highly conserved...
3999     B  thermoanaerobacter tengcongensis is a rod-shap...

[4000 rows x 2 columns]
                                              abstract
0    in a previous work all three components of com...
1    we compared morphology of two geographically c...
2    factor xiii mr 320000 is a blood coagulation f...
3    we rep

## Break Down abstract 

Break Down abstract into seperate words so we can analyse the frequency of words. First we will find all the words present.

I will use the NLTK library to do this. From the NLTK library I will be using the tokenizing funtion (break down each word) and removing stopping words using the list of words from the library and finally counting the number of words using the FreqDist Class.

In [4]:
allWords = ""

for index in range(len(absTrainData)):
  abstract = absTrainData["abstract"][index]
  # print(type(abstract))
  allWords += "\n" + abstract



# print(allWords)

allWordsTokens = word_tokenize(allWords)

# print("Tokens before filter: ")

# print(*allWordsTokens, sep= "\n")

filteredWords = [word for word in allWordsTokens if word.casefold() not in stop_words]

# print(len(filteredWords))

# print("Tokens after filter: ")
# print(*filteredWords, sep= "\n")

uniqueWords = set(filteredWords)
# print(len(uniqueWords))

fD = FreqDist(filteredWords)
# print(len(frequncyDistribution))
for word in fD.items():
   print(word[0])

4
202
353
bp
genome
alkaliphilic
bacterium
bacillus
halodurans
c-125
contains
4066
predicted
protein
coding
sequences
cdss
2141
527
functional
assignments
1182
29
conserved
unknown
function
743
18
3
match
database
among
total
88
proteins
found
subtilis
667
widely
comparison
various
organisms
including
bsubtilis
b
112
transposase
genes
indicating
transposases
played
important
evolutionary
role
horizontal
gene
transfer
also
internal
genetic
rearrangement
strain
lacks
necessary
competence
coms
srfa
rapc
supporting
fact
demonstrated
experimentally
paralog
tupa
encoding
teichuronopeptide
contributes
alkaliphily
ortholog
11
sigma
factors
belong
extracytoplasmic
family
10
unique
suggesting
may
special
mechanism
adaptation
alkaline
environment
complete
1751377-bp
sequence
thermophilic
archaeon
methanobacterium
thermoautotrophicum
deltah
determined
whole-genome
shotgun
sequencing
approach
1855
open
reading
frames
orfs
identified
appear
encode
polypeptides
844
46
assigned
putative
functions
base

## Creating the functions

In [82]:
import multiprocessing

def lock_init(main_lock):

    global lock
    lock = main_lock


class probVars():

    def __init__(self, cls, cls_p, numOfUniqueWords, filtered, freq_filtered) -> None:
        self.cls = cls
        self.cls_p = cls_p
        self.numOfUniqueWords = numOfUniqueWords
        self.filtered = filtered
        self.freq_filtered = freq_filtered

    def load(self):
        return self.cls, self.cls_p, self.numOfUniqueWords, self.filtered, self.freq_filtered


class BayesianModel:
    

    def __init__(self, training_data: DataFrame, class_variables) -> None:
        self.class_variables = class_variables
        self.training_data = training_data

        self.posterior_probability = {}
        self.word_frequncy = {}
        self.class_totals = {}
        for cls in class_variables:
            self.posterior_probability[cls] = 0
            self.word_frequncy[cls] = {}
            self.class_totals[cls] = 0
        self.needLaplaceSmoothing = False
        self.uniqueWords = set()
        self.calculate_posterior_probabilities()
        self.count_words()


    def count_words(self):
        data = self.training_data

        
        for cls in self.class_variables:
            wordsInClass = []
            for index in range(len(data)):
                if cls == data["class"][index]:
                    abstract = data["abstract"][index]
                    wordsInClass += word_tokenize(abstract)
            filteredInClass = [word for word in wordsInClass if word.casefold() not in stop_words]
            totalTokensInClass = len(filteredInClass)
            frequncyInClass = FreqDist(filteredInClass)
            for word in frequncyInClass.items():
                self.uniqueWords.add(word[0])
                self.word_frequncy[cls][word[0]] = word[1] + 1
                self.class_totals[cls] = totalTokensInClass
            # print(self.word_frequncy[cls])
        print("counted all")


        # data = self.training_data
        # allWords = ""
        # for index in range(len(data)):
        #     abstract = data["abstract"][index]
        #     allWords += "\n" + abstract
        
        # allTokens = word_tokenize(allWords)
        # filteredWords = [word for word in allTokens if word.casefold() not in stop_words]

        # uniqueWords = set(filteredWords)


        # self.needLaplaceSmoothing = self.need_laplace(uniqueWords)

        # wordFrequncy = FreqDist(filteredWords)
        
    def need_laplace(self, uniqueWords):
        data = self.training_data

        for cls in self.class_variables:
            wordsInClass = ""
            for index in range(len(data)):
                if cls == data["class"][index]:
                    abstract = data["abstract"][index]
                    wordsInClass += "\n" + abstract
            tokensInClass = word_tokenize(wordsInClass)
            filteredInClass = [word for word in tokensInClass if word.casefold() not in stop_words]
            uniqueInClass = set(filteredInClass)
            if len(uniqueInClass) != len(uniqueWords):
                return True

        return False

    def test(self, test_data: DataFrame) -> list:
        # result list to return
        results = []

        # go through each abstract in the test_data seperately
        for index in range(len(test_data)):

            # copy the dictionary --> previously took the point instead
            cls_p = self.posterior_probability.copy()

            # get abstract and tokenise and remove stop words
            abstract = test_data["abstract"][index]
            tokens = word_tokenize(abstract)
            filtered = [word for word in tokens if word.casefold() not in stop_words]
            freq_filtered = FreqDist(filtered)

            # get the unique words from the filtered list
            set_of_filtered = set(filtered)

            # use union operation to combine sets to find total new words for laplace smoothing
            all_words = set_of_filtered.union(self.uniqueWords)
            numOfUniqueWords = len(all_words)

            # seperate this and use parallel computing to speed things up --> nope not sure whats wrong...

            # pVs = []
            for cls in self.class_variables:
                pV = probVars(cls, cls_p, numOfUniqueWords, filtered, freq_filtered)
                self.calc_prob(pV)
                # pVs.append(pV)
            

            # # p = multiprocessing.Pool(processes=len(self.class_variables), initializer=lock_init, initargs=self.lock)
            # print("starting processors")
            # # p.map(self.calc_prob, pVs)

            # p1 = multiprocessing.Process(target=self.calc_prob, args=(pVs[0], self.lock))
            # p2 = multiprocessing.Process(target=self.calc_prob, args=(pVs[1], self.lock))
            # p3 = multiprocessing.Process(target=self.calc_prob, args=(pVs[2], self.lock))
            # p4 = multiprocessing.Process(target=self.calc_prob, args=(pVs[3], self.lock))

            # p1.start()
            # p2.start()
            # p3.start()
            # p4.start()

            # p1.join()
            # p2.join()
            # p3.join()
            # p4.join()


            
            # always do a laplace smoothing as it only makes things better
            # for cls in self.class_variables:
            #     cls_p[cls] = log(cls_p[cls])
            #     cls_t = self.class_totals.copy()
            #     cls_t[cls] += (len(cls_p)) * numOfUniqueWords
            #     word_frequncy:dict = self.word_frequncy[cls].copy()
            #     for word in set(filtered).difference(set(list(word_frequncy.keys()))):
            #             word_frequncy[word] = 1

            #     # could make this faster with the frequncy dist and then add it that many times instead of looping the entire list --> done
            #     for w_n in freq_filtered.items():
            #         cls_p[cls] += w_n[1] * (log(word_frequncy[w_n[0]]/cls_t[cls]))

            cls_prediction = list(cls_p.keys())[0]
            for cls in list(cls_p.keys()):
                if cls_p[cls] > cls_p[cls_prediction]:
                    cls_prediction = cls
            results.append(cls_prediction)

        return results

    def calculate_posterior_probabilities(self):
        data = self.training_data
        total = len(data)
        for cls in self.class_variables:
            self.posterior_probability[cls] = len(data[data['class'] == cls])/total
        print(self.posterior_probability)

    def calc_prob(self, pV: probVars):
        cls, cls_p, numOfUniqueWords, filtered, freq_filtered = pV.load()
        cls_p[cls] = log(cls_p[cls])
        cls_t = self.class_totals.copy()
        cls_t[cls] += (len(cls_p)) * numOfUniqueWords
        word_frequncy:dict = self.word_frequncy[cls].copy()
        for word in set(filtered).difference(set(list(word_frequncy.keys()))):
            word_frequncy[word] = 1

            # could make this faster with the frequncy dist and then add it that many times instead of looping the entire list --> done
        for w_n in freq_filtered.items():
            wF = word_frequncy[w_n[0]]/cls_t[cls]
            p = w_n[1]
            cls_p[cls] += p * (log(wF))


## Test with some data so we can validate model

In [83]:
# Train with some data

def single_split_validation(
    data_set: DataFrame,
    training_validation_split=0.5,
):
    training_data = data_set.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=training_validation_split, random_state= 200))
    validation_data = data_set.drop(training_data.index)
    training_data = training_data.reset_index(drop=True)
    validation_data = validation_data.reset_index(drop=True)
    BM = BayesianModel(training_data, classKeys)
    predictions = BM.test(validation_data)
    total = len(validation_data)
    correct = 0
    for i in range(len(validation_data)):
        if predictions[i] == validation_data['class'][i]:
            correct+=1
    print(f'{correct/total}')

# 90/10 split
single_split_validation(absTrainData, training_validation_split= 0.9)

# 70/30 split
single_split_validation(absTrainData, training_validation_split= 0.7)

# 50/50 split
single_split_validation(absTrainData, training_validation_split= 0.5)

C:\Users\liyor\AppData\Local\Temp\ipykernel_13696\571344429.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  training_data = data_set.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=training_validation_split, random_state= 200))


{'A': 0.03194444444444444, 'B': 0.40055555555555555, 'E': 0.5361111111111111, 'V': 0.03138888888888889}
counted all
{'4': 2.855970331178832, '202': 5.298317366548036, '353': 5.298317366548036, 'bp': 1.984131361875511, 'genome': 1.0078579253996456, 'alkaliphilic': 4.605170185988092, 'bacterium': 2.436116485618568, 'bacillus': 2.8134107167600364, 'halodurans': 4.892852258439873, 'c-125': 5.298317366548036, 'contains': 1.3280254529959148, '4066': 4.892852258439873, 'predicted': 1.6873994539038122, 'protein': 0.843970070294529, 'coding': 1.8643301620628903, 'sequences': 1.0010319603292457, 'cdss': 4.382026634673881, '2141': 5.298317366548036, '527': 4.892852258439873, 'functional': 2.436116485618568, 'assignments': 4.605170185988092, '1182': 5.298317366548036, '29': 4.605170185988092, 'conserved': 1.786771927717016, 'unknown': 2.995732273553991, 'function': 1.8805906829346708, '743': 5.298317366548036, '18': 3.7942399697717626, '3': 2.4651040224918206, 'match': 5.298317366548036, 'database

C:\Users\liyor\AppData\Local\Temp\ipykernel_13696\571344429.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  training_data = data_set.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=training_validation_split, random_state= 200))


counted all
{'4': 2.772588722239781, '202': 5.298317366548036, '353': 5.480638923341991, 'bp': 1.9367852412783126, 'genome': 1.0617983155453934, 'alkaliphilic': 4.787491742782046, 'bacterium': 2.50510935710552, 'bacillus': 2.772588722239781, 'halodurans': 5.144166686720778, 'c-125': 5.298317366548036, 'contains': 1.455287232606842, '4066': 5.298317366548036, 'predicted': 1.5686159179138452, 'protein': 0.8635401664886262, 'coding': 1.8430527636156055, 'sequences': 1.030953640194295, 'cdss': 4.692181562977721, '2141': 5.480638923341991, '527': 5.0106352940962555, 'functional': 2.4266377416640244, 'assignments': 4.605170185988092, '1182': 5.480638923341991, '29': 3.9990343824177756, 'conserved': 1.7670668566376835, 'unknown': 2.900422093749666, 'function': 1.8536348729461425, '743': 5.480638923341991, '18': 3.452490676049706, '3': 2.3626890170637513, 'match': 5.144166686720778, 'database': 3.3765047690717838, 'among': 2.21487951257494, 'total': 2.70805020110221, '88': 4.5251274783145545, 

C:\Users\liyor\AppData\Local\Temp\ipykernel_13696\571344429.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  training_data = data_set.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=training_validation_split, random_state= 200))


counted all
{'4': 2.8473122684357177, '202': 5.654992310486769, '353': 5.654992310486769, 'bp': 1.9519282213808764, 'genome': 1.0469690555162712, 'alkaliphilic': 5.115995809754082, 'bacterium': 2.436116485618568, 'bacillus': 2.873514640829742, 'halodurans': 5.403677882205863, 'c-125': 5.403677882205863, 'contains': 1.4631754054558481, '4066': 5.298317366548036, 'predicted': 1.616966178854892, 'protein': 0.8867319296326106, 'coding': 1.8078888511579387, 'sequences': 1.0202633222571333, 'cdss': 4.767689115485866, '2141': 5.809142990314028, '527': 5.2030071867437115, 'functional': 2.35915544448244, 'assignments': 4.892852258439873, '1182': 5.809142990314028, '29': 3.937340813412436, 'conserved': 1.789761466565382, 'unknown': 2.900422093749666, 'function': 1.8420006856648017, '743': 5.809142990314028, '18': 3.490028595368771, '3': 2.4304184645039304, 'match': 5.115995809754082, 'database': 3.270169119255751, 'among': 2.1803674602697964, 'total': 2.6664285264113907, '88': 4.8283137373023015

## Test with all data once found best hyperparameters

In [84]:
# Train with all data
BM = BayesianModel(absTrainData, classKeys)

predictions = BM.test(absTestData)

results = pd.read_csv("results/946prediction.csv")
total = len(results)
correct =0
for i in range(len(results)):
    if predictions[i] == results['class'][i]:
        correct += 1
print(f"{correct/total}")

file = open("results/predictions.csv", "w")

s = "class,id\n"

id = 1
for p in predictions:
    s+=f"{p},{id}\n"
    id+=1

file.write(s)
file.close()

{'A': 0.032, 'B': 0.4005, 'E': 0.536, 'V': 0.0315}
counted all
{'previous': 3.540459448995663, 'work': 4.268697949366879, 'three': 1.6044503709230613, 'components': 2.9759296462578115, 'comamonas': 6.214608098422191, 'testosteroni': 5.809142990314028, 'b-356': 6.214608098422191, 'biphenyl': 5.809142990314028, 'bphchlorobiphenyls': 6.907755278982137, 'pcbs': 6.907755278982137, 'dioxygenase': 6.214608098422191, 'dox': 6.907755278982137, 'purified': 2.563949857128453, 'characterized': 2.3538783873815965, 'include': 2.900422093749666, 'iron-sulphur': 6.214608098422191, 'protein': 0.8462983600541202, 'ispbph': 6.907755278982137, 'terminal': 4.135166556742356, 'oxygenase': 6.907755278982137, 'composed': 3.2441936328524905, 'two': 0.8533159327127667, 'subunits': 2.9565115604007097, 'encoded': 2.162823150618887, 'bpha': 6.907755278982137, 'bphe': 6.907755278982137, 'ferredoxin': 4.8283137373023015, 'ferbph': 6.907755278982137, 'bphf': 6.907755278982137, 'reductase': 3.9633162998156966, 'redbph

## Add word weights

In [ ]:
import multiprocessing

def lock_init(main_lock):

    global lock
    lock = main_lock


class probVars():

    def __init__(self, cls, cls_p, numOfUniqueWords, filtered, freq_filtered) -> None:
        self.cls = cls
        self.cls_p = cls_p
        self.numOfUniqueWords = numOfUniqueWords
        self.filtered = filtered
        self.freq_filtered = freq_filtered

    def load(self):
        return self.cls, self.cls_p, self.numOfUniqueWords, self.filtered, self.freq_filtered


class BayesianModel:
    

    def __init__(self, training_data: DataFrame, class_variables) -> None:
        self.class_variables = class_variables
        self.training_data = training_data

        self.posterior_probability = {}
        self.word_frequncy = {}
        self.word_weights = {}
        self.class_totals = {}
        for cls in class_variables:
            self.posterior_probability[cls] = 0
            self.word_frequncy[cls] = {}
            self.class_totals[cls] = 0
        self.needLaplaceSmoothing = False
        self.uniqueWords = set()
        self.calculate_posterior_probabilities()
        self.count_words()


    def count_words(self):
        data = self.training_data

        
        for cls in self.class_variables:
            wordsInClass = []
            for index in range(len(data)):
                if cls == data["class"][index]:
                    abstract = data["abstract"][index]
                    wordsInClass += word_tokenize(abstract)
            filteredInClass = [word for word in wordsInClass if word.casefold() not in stop_words]
            totalTokensInClass = len(filteredInClass)
            frequncyInClass = FreqDist(filteredInClass)
            for word in frequncyInClass.items():
                self.uniqueWords.add(word[0])
                self.word_frequncy[cls][word[0]] = word[1] + 1
                self.class_totals[cls] = totalTokensInClass
            # print(self.word_frequncy[cls])
            # print(self.word_weights[cls])
        print("counted all")


        # data = self.training_data
        # allWords = ""
        # for index in range(len(data)):
        #     abstract = data["abstract"][index]
        #     allWords += "\n" + abstract
        
        # allTokens = word_tokenize(allWords)
        # filteredWords = [word for word in allTokens if word.casefold() not in stop_words]

        # uniqueWords = set(filteredWords)


        # self.needLaplaceSmoothing = self.need_laplace(uniqueWords)

        # wordFrequncy = FreqDist(filteredWords)
        
    def need_laplace(self, uniqueWords):
        data = self.training_data

        for cls in self.class_variables:
            wordsInClass = ""
            for index in range(len(data)):
                if cls == data["class"][index]:
                    abstract = data["abstract"][index]
                    wordsInClass += "\n" + abstract
            tokensInClass = word_tokenize(wordsInClass)
            filteredInClass = [word for word in tokensInClass if word.casefold() not in stop_words]
            uniqueInClass = set(filteredInClass)
            if len(uniqueInClass) != len(uniqueWords):
                return True

        return False

    def test(self, test_data: DataFrame) -> list:
        # result list to return
        results = []

        tokensList = [set() for i in range(len(test_data))]
        allWords = []

        for index in range(len(test_data)):
            abstract = test_data["abstract"][index]
            tokens = word_tokenize(abstract)
            tokens = [word for word in tokens if word.casefold() not in stop_words]
            allWords += tokens
            tokensList[index] = set(tokens)
        
        for word in allWords:
            occurances = 0
            for tokens in tokensList:
                if tokens.__contains__(word):
                    occurances += 1
            if occurances == 0:
                # print(word)
                occurances = 1
            self.word_weights[word] = log(len(test_data)/occurances)
        print(self.word_weights)

        # go through each abstract in the test_data seperately
        for index in range(len(test_data)):

            # copy the dictionary --> previously took the point instead
            cls_p = self.posterior_probability.copy()

            # get abstract and tokenise and remove stop words
            abstract = test_data["abstract"][index]
            tokens = word_tokenize(abstract)
            filtered = [word for word in tokens if word.casefold() not in stop_words]
            freq_filtered = FreqDist(filtered)

            # get the unique words from the filtered list
            set_of_filtered = set(filtered)

            # use union operation to combine sets to find total new words for laplace smoothing
            all_words = set_of_filtered.union(self.uniqueWords)
            numOfUniqueWords = len(all_words)

            



            # seperate this and use parallel computing to speed things up --> nope not sure whats wrong...

            # pVs = []
            for cls in self.class_variables:
                pV = probVars(cls, cls_p, numOfUniqueWords, filtered, freq_filtered)
                self.calc_prob(pV)
                # pVs.append(pV)
            

            # # p = multiprocessing.Pool(processes=len(self.class_variables), initializer=lock_init, initargs=self.lock)
            # print("starting processors")
            # # p.map(self.calc_prob, pVs)

            # p1 = multiprocessing.Process(target=self.calc_prob, args=(pVs[0], self.lock))
            # p2 = multiprocessing.Process(target=self.calc_prob, args=(pVs[1], self.lock))
            # p3 = multiprocessing.Process(target=self.calc_prob, args=(pVs[2], self.lock))
            # p4 = multiprocessing.Process(target=self.calc_prob, args=(pVs[3], self.lock))

            # p1.start()
            # p2.start()
            # p3.start()
            # p4.start()

            # p1.join()
            # p2.join()
            # p3.join()
            # p4.join()


            
            # always do a laplace smoothing as it only makes things better
            # for cls in self.class_variables:
            #     cls_p[cls] = log(cls_p[cls])
            #     cls_t = self.class_totals.copy()
            #     cls_t[cls] += (len(cls_p)) * numOfUniqueWords
            #     word_frequncy:dict = self.word_frequncy[cls].copy()
            #     for word in set(filtered).difference(set(list(word_frequncy.keys()))):
            #             word_frequncy[word] = 1

            #     # could make this faster with the frequncy dist and then add it that many times instead of looping the entire list --> done
            #     for w_n in freq_filtered.items():
            #         cls_p[cls] += w_n[1] * (log(word_frequncy[w_n[0]]/cls_t[cls]))

            cls_prediction = list(cls_p.keys())[0]
            for cls in list(cls_p.keys()):
                if cls_p[cls] > cls_p[cls_prediction]:
                    cls_prediction = cls
            results.append(cls_prediction)

        return results

    def calculate_posterior_probabilities(self):
        data = self.training_data
        total = len(data)
        for cls in self.class_variables:
            self.posterior_probability[cls] = len(data[data['class'] == cls])/total
        print(self.posterior_probability)

    def calc_prob(self, pV: probVars):
        cls, cls_p, numOfUniqueWords, filtered, freq_filtered = pV.load()
        cls_p[cls] = log(cls_p[cls])
        cls_t = self.class_totals.copy()
        cls_t[cls] += (len(cls_p)) * numOfUniqueWords
        word_frequncy:dict = self.word_frequncy[cls].copy()
        word_weights:dict = self.word_weights
        for word in set(filtered).difference(set(list(word_frequncy.keys()))):
            word_frequncy[word] = 1

            # could make this faster with the frequncy dist and then add it that many times instead of looping the entire list --> done
        for w_n in freq_filtered.items():
            weight = word_weights[w_n[0]]
            wF = word_frequncy[w_n[0]]/cls_t[cls]
            p = weight * w_n[1]
            cls_p[cls] += p * (log(wF))
